[http://fastml.com/tuning-hyperparams-fast-with-hyperband/](http://fastml.com/tuning-hyperparams-fast-with-hyperband/)

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
import numpy as np
from hyperband import Hyperband

In [2]:
def load_mnist():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()
    train_X = train_X.reshape(-1, 784)
    test_X = test_X.reshape(-1, 784)
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')
    train_X /= 255
    test_X /= 255

    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return {"train_X": train_X[:55000, :], "train_y": train_y[:55000, :],
            "val_X": train_X[55000:, :], "val_y": train_y[55000:, :], "test_X": test_X, "test_y": test_y}

In [3]:
def build_network(keep_prob=0.5, optimizer='adam'):
    inputs = Input(shape=(784,), name="input")
    x = Dense(512, activation='relu', name="hidden1")(inputs)
    x = Dropout(keep_prob)(x)
    x = Dense(256, activation='relu', name="hidden2")(x)
    x = Dropout(keep_prob)(x)
    x = Dense(128, activation='relu', name="hidden3")(x)
    x = Dropout(keep_prob)(x)
    prediction = Dense(10, activation='softmax', name="output")(x)
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    return model

In [4]:
def get_params():
    batches = np.random.choice([5, 10, 100])
    optimizers = np.random.choice(['rmsprop', 'adam', 'adadelta'])
    dropout = np.random.choice(np.linspace(0.1, 0.5, 10))
    return {"batch_size": batches, "optimizer": optimizers, "keep_prob": dropout}

In [5]:
def try_params(data, num_iters, hyperparameters):
    model = build_network(keep_prob=hyperparameters["keep_prob"], optimizer=hyperparameters["optimizer"])
    model.fit(x=data["train_X"], y=data["train_y"],
              batch_size=hyperparameters["batch_size"],
              epochs=int(num_iters))
    loss = model.evaluate(x=data["val_X"], y=data["val_y"], verbose=0)

    return {"loss": loss}

In [6]:
def main():
    data = load_mnist()
    hb = Hyperband(data, get_params, try_params)
    results = hb.run()
    print(results)

In [7]:
main()


*** 81 configurations x 1.0 iterations each

1 | Fri Oct  8 13:38:43 2021 | lowest loss so far: inf (run -1)

11000/11000 [==============================] - 62s 6ms/step - loss: 2.1763

63 seconds.

2 | Fri Oct  8 13:39:46 2021 | lowest loss so far: 1.9459 (run 1)

11000/11000 [==============================] - 66s 6ms/step - loss: 2.1762

67 seconds.

3 | Fri Oct  8 13:40:52 2021 | lowest loss so far: 1.9459 (run 1)

5500/5500 [==============================] - 41s 7ms/step - loss: 0.3065

42 seconds.

4 | Fri Oct  8 13:41:34 2021 | lowest loss so far: 0.1751 (run 3)

550/550 [==============================] - 5s 8ms/step - loss: 2.3706

5 seconds.

5 | Fri Oct  8 13:41:39 2021 | lowest loss so far: 0.1751 (run 3)

5500/5500 [==============================] - 23s 4ms/step - loss: 2.2183

23 seconds.

6 | Fri Oct  8 13:42:03 2021 | lowest loss so far: 0.1751 (run 3)

5500/5500 [==============================] - 19s 3ms/step - loss: 0.2398

20 seconds.

7 | Fri Oct  8 13:42:22 2021 | l

KeyboardInterrupt: 